## English version

In [ ]:
import random
import sys
import threading

sys.setrecursionlimit(100000)
threading.stack_size(1 << 25)

class GoGame:
    def __init__(self, size=9):  # Default board size is set to 9x9
        self.size = size
        self.board = [['.' for _ in range(size)] for _ in range(size)]
        self.current_player = 'B'  # 'B' represents Black stones, 'W' represents White stones
        self.captured_stones = {'B': 0, 'W': 0}
        self.game_over = False

    def print_board(self):
        # Display the board with coordinates
        header = '   ' + ' '.join(f'{i+1:2}' for i in range(self.size))
        print(header)
        for idx, row in enumerate(self.board):
            display_row = []
            for cell in row:
                if cell == 'B':
                    display_cell = '●'
                elif cell == 'W':
                    display_cell = '○'
                else:
                    display_cell = '・'  # Use a dot for empty points
                display_row.append(display_cell)
            print(f"{idx+1:2} {' '.join(display_row)}")
        print()

    def place_stone(self, x, y):
        x -= 1  # Adjust for 0-based indexing
        y -= 1
        if self.board[x][y] != '.':
            print("Invalid move: position already occupied.")
            return False
        self.board[x][y] = self.current_player
        if not self.check_and_capture(x, y):
            self.board[x][y] = '.'
            print("Suicide move is not allowed.")
            return False
        self.current_player = 'W' if self.current_player == 'B' else 'B'
        return True

    def model_move(self):
        valid_moves = self.get_valid_moves('W')
        if valid_moves:
            x, y = random.choice(valid_moves)
            print(f"Computer (Player ○) placed a stone at: {x+1} {y+1}")
            self.place_stone(x+1, y+1)
        else:
            print("Computer (Player ○) has no valid moves.")
            self.end_game()

    def check_and_capture(self, x, y):
        player = self.board[x][y]
        opponent = 'W' if player == 'B' else 'B'
        captured_opponent_stones = set()

        # Check adjacent opponent stones for capture
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < self.size
                and 0 <= ny < self.size
                and self.board[nx][ny] == opponent
            ):
                group = set()
                if self.is_captured(nx, ny, opponent, group, set()):
                    captured_opponent_stones.update(group)

        # Remove captured opponent stones
        for gx, gy in captured_opponent_stones:
            self.board[gx][gy] = '.'
        self.captured_stones[player] += len(captured_opponent_stones)

        # Check if placing the stone results in self-capture
        group = set()
        if self.is_captured(x, y, player, group, set()):
            if len(captured_opponent_stones) == 0:
                return False  # Suicide move is invalid
            else:
                # Remove own stones if captured
                for gx, gy in group:
                    self.board[gx][gy] = '.'
                self.captured_stones[opponent] += len(group)
        return True

    def is_captured(self, x, y, player, group, visited):
        if (x, y) in visited:
            return True  # Already checked
        visited.add((x, y))

        if self.board[x][y] == '.':
            return False  # Has liberty
        if self.board[x][y] != player:
            return True  # Blocked by opponent

        group.add((x, y))

        # Recursively check adjacent positions
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < self.size
                and 0 <= ny < self.size
                and not self.is_captured(nx, ny, player, group, visited)
            ):
                return False  # Has liberty
        return True  # No liberties

    def remove_dead_stones(self):
        # Identify and remove dead stones
        dead_stones = {'B': set(), 'W': set()}
        for player in ['B', 'W']:
            opponent = 'W' if player == 'B' else 'B'
            for x in range(self.size):
                for y in range(self.size):
                    if self.board[x][y] == player:
                        if not self.is_alive(x, y, player, set()):
                            dead_stones[opponent].add((x, y))
        for player in ['B', 'W']:
            for x, y in dead_stones[player]:
                self.board[x][y] = '.'
            self.captured_stones[player] += len(dead_stones[player])

    def is_alive(self, x, y, player, visited):
        if (x, y) in visited:
            return False
        visited.add((x, y))

        # Check for two eyes
        eyes = self.count_eyes(x, y, player, set())
        if eyes >= 2:
            return True

        # Check for liberties
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if self.board[nx][ny] == '.':
                    return True
                elif self.board[nx][ny] == player:
                    if self.is_alive(nx, ny, player, visited):
                        return True
        return False

    def count_eyes(self, x, y, player, visited):
        if (x, y) in visited:
            return 0
        visited.add((x, y))
        eyes = 0
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if self.board[nx][ny] == '.':
                    if self.is_eye(nx, ny, player, set()):
                        eyes += 1
                elif self.board[nx][ny] == player:
                    eyes += self.count_eyes(nx, ny, player, visited)
        return eyes

    def is_eye(self, x, y, player, visited):
        if (x, y) in visited:
            return True
        visited.add((x, y))
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if self.board[nx][ny] != player:
                    return False
        return True

    def calculate_territory(self):
        territory = {'B': 0, 'W': 0}
        checked = set()
        for x in range(self.size):
            for y in range(self.size):
                if self.board[x][y] == '.' and (x, y) not in checked:
                    owners, area = self.find_owners(x, y, checked)
                    if len(owners) == 1:
                        owner = owners.pop()
                        territory[owner] += area
        return territory

    def find_owners(self, x, y, checked):
        owners = set()
        area = 0
        stack = [(x, y)]
        while stack:
            cx, cy = stack.pop()
            if (cx, cy) in checked:
                continue
            checked.add((cx, cy))
            area += 1
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = cx + dx, cy + dy
                if 0 <= nx < self.size and 0 <= ny < self.size:
                    if self.board[nx][ny] == '.':
                        stack.append((nx, ny))
                    else:
                        owners.add(self.board[nx][ny])
        return owners, area

    def has_valid_moves(self, player):
        for x in range(self.size):
            for y in range(self.size):
                if self.board[x][y] == '.' and self.is_valid_move(x, y, player):
                    return True
        return False

    def get_valid_moves(self, player):
        valid_moves = []
        for x in range(self.size):
            for y in range(self.size):
                if self.board[x][y] == '.' and self.is_valid_move(x, y, player):
                    valid_moves.append((x, y))
        return valid_moves

    def is_valid_move(self, x, y, player):
        # Temporarily place the stone and check for validity
        self.board[x][y] = player
        is_valid = True
        if not self.check_and_capture_for_validation(x, y):
            is_valid = False
        self.board[x][y] = '.'  # Revert the move
        return is_valid

    def check_and_capture_for_validation(self, x, y):
        player = self.board[x][y]
        opponent = 'W' if player == 'B' else 'B'
        captured_opponent_stones = set()

        # Check for capture of opponent stones
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < self.size
                and 0 <= ny < self.size
                and self.board[nx][ny] == opponent
            ):
                group = set()
                if self.is_captured(nx, ny, opponent, group, set()):
                    captured_opponent_stones.update(group)

        # Temporarily remove captured opponent stones
        for gx, gy in captured_opponent_stones:
            self.board[gx][gy] = '.'

        # Check if own stone is captured
        group = set()
        result = True
        if self.is_captured(x, y, player, group, set()):
            if len(captured_opponent_stones) == 0:
                result = False  # Suicide move is invalid

        # Revert the temporary removal of opponent stones
        for gx, gy in captured_opponent_stones:
            self.board[gx][gy] = opponent

        return result

    def play(self):
        # Set the board size
        size_input = input(
            "Enter board size (e.g., 9, 13, 19) [Default: 9]: "
        )
        if size_input.isdigit():
            self.size = int(size_input)
            self.board = [['.' for _ in range(self.size)] for _ in range(self.size)]
        else:
            print("Starting game with default size 9.")
        while not self.game_over:
            if not self.has_valid_moves(self.current_player):
                print(
                    f"Player {self.get_player_symbol(self.current_player)} has no valid moves."
                )
                self.end_game()
                break

            self.print_board()
            print(
                f"Captured Stones: ●={self.captured_stones['B']}, ○={self.captured_stones['W']}"
            )
            if self.current_player == 'B':
                try:
                    move = input(
                        f"Player {self.get_player_symbol(self.current_player)}, enter your move (row column): "
                    )
                    if move.lower() == 'quit':
                        print("Ending the game.")
                        self.end_game()
                        break
                    x, y = map(int, move.strip().split())
                    if x < 1 or x > self.size or y < 1 or y > self.size:
                        print("Invalid move: out of bounds.")
                        continue
                    if not self.place_stone(x, y):
                        continue
                except ValueError:
                    print("Invalid input: please enter numbers for row and column.")
            else:
                self.model_move()

    def get_player_symbol(self, player):
        return '●' if player == 'B' else '○'

    def end_game(self):
        # 1. Remove dead stones
        self.remove_dead_stones()

        # 2. Display the scoring board
        print("Scoring Board:")
        self.print_board()

        # 3. Calculate territories for both players
        territory = self.calculate_territory()

        # 4. Calculate the total score
        total_score = {
            'B': territory['B'] + self.captured_stones['B'],
            'W': territory['W'] + self.captured_stones['W'],
        }

        # 5. Display the scores
        print("Captured Stones:")
        print(f"●: {self.captured_stones['B']} stones")
        print(f"○: {self.captured_stones['W']} stones")
        print("Territory:")
        print(f"●: {territory['B']} points")
        print(f"○: {territory['W']} points")
        print("Final Score:")
        print(f"●: {total_score['B']} points")
        print(f"○: {total_score['W']} points")

        # 6. Determine the winner
        if total_score['B'] > total_score['W']:
            print("Player ● wins!")
        elif total_score['B'] < total_score['W']:
            print("Player ○ wins!")
        else:
            print("The game is a draw!")
        self.game_over = True

if __name__ == "__main__":
    game = GoGame()
    game.play()


Enter board size (e.g., 9, 13, 19) [Default: 9]: 5
    1  2  3  4  5
 1 ・ ・ ・ ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ・ ・ ・
 5 ・ ・ ・ ・ ・

Captured Stones: ●=0, ○=0
Player ●, enter your move (row column): 2 2
    1  2  3  4  5
 1 ・ ・ ・ ・ ・
 2 ・ ● ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ・ ・ ・
 5 ・ ・ ・ ・ ・

Captured Stones: ●=0, ○=0
Computer (Player ○) placed a stone at: 5 2
    1  2  3  4  5
 1 ・ ・ ・ ・ ・
 2 ・ ● ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ・ ・ ・
 5 ・ ○ ・ ・ ・

Captured Stones: ●=0, ○=0
Player ●, enter your move (row column): 1 2
    1  2  3  4  5
 1 ・ ● ・ ・ ・
 2 ・ ● ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ・ ・ ・
 5 ・ ○ ・ ・ ・

Captured Stones: ●=0, ○=0
Computer (Player ○) placed a stone at: 3 4
    1  2  3  4  5
 1 ・ ● ・ ・ ・
 2 ・ ● ・ ・ ・
 3 ・ ・ ・ ○ ・
 4 ・ ・ ・ ・ ・
 5 ・ ○ ・ ・ ・

Captured Stones: ●=0, ○=0
Player ●, enter your move (row column): 2 4
    1  2  3  4  5
 1 ・ ● ・ ・ ・
 2 ・ ● ・ ● ・
 3 ・ ・ ・ ○ ・
 4 ・ ・ ・ ・ ・
 5 ・ ○ ・ ・ ・

Captured Stones: ●=0, ○=0
Computer (Player ○) placed a stone at: 2 1
    1  2  3  4  5
 1 ・ ● ・ ・

## Japanese version

In [ ]:
import random
import sys
import threading

sys.setrecursionlimit(100000)
threading.stack_size(1 << 25)

class GoGame:
    def __init__(self, size=9):  # デフォルトの盤面サイズを9x9に設定
        self.size = size
        self.board = [['.' for _ in range(size)] for _ in range(size)]
        self.current_player = 'B'  # 'B'は黒石、'W'は白石を表します
        self.captured_stones = {'B': 0, 'W': 0}
        self.game_over = False

    def print_board(self):
        # 横軸の数字を表示
        header = '   ' + ' '.join(f'{i+1:2}' for i in range(self.size))
        print(header)
        for idx, row in enumerate(self.board):
            display_row = []
            for cell in row:
                if cell == 'B':
                    display_cell = '●'
                elif cell == 'W':
                    display_cell = '○'
                else:
                    display_cell = '・'  # 全角の中点
                display_row.append(display_cell)
            # 縦軸の数字を表示
            print(f"{idx+1:2} {' '.join(display_row)}")
        print()

    def place_stone(self, x, y):
        x -= 1  # インデックスは0から始まるため調整
        y -= 1
        if self.board[x][y] != '.':
            print("無効な手です。その位置はすでに埋まっています。")
            return False
        self.board[x][y] = self.current_player
        if not self.check_and_capture(x, y):
            self.board[x][y] = '.'
            print("自殺手は無効です。")
            return False
        self.current_player = 'W' if self.current_player == 'B' else 'B'
        return True

    def model_move(self):
        valid_moves = self.get_valid_moves('W')
        if valid_moves:
            x, y = random.choice(valid_moves)
            print(f"モデル（プレイヤー ○）が石を置きました: {x+1} {y+1}")
            self.place_stone(x+1, y+1)
        else:
            print("モデル（プレイヤー ○）は有効な手がありません。")
            self.end_game()

    def check_and_capture(self, x, y):
        player = self.board[x][y]
        opponent = 'W' if player == 'B' else 'B'
        captured_opponent_stones = set()

        # 隣接する相手の石のグループをチェック
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size and self.board[nx][ny] == opponent:
                group = set()
                if self.is_captured(nx, ny, opponent, group, set()):
                    captured_opponent_stones.update(group)

        # 捕獲した相手の石を取り除く
        for gx, gy in captured_opponent_stones:
            self.board[gx][gy] = '.'
        self.captured_stones[player] += len(captured_opponent_stones)

        # 自分の石が捕獲されるかチェック
        group = set()
        if self.is_captured(x, y, player, group, set()):
            if len(captured_opponent_stones) == 0:
                return False  # 自殺手は無効
            else:
                # 自分の石を取り除く
                for gx, gy in group:
                    self.board[gx][gy] = '.'
                self.captured_stones[opponent] += len(group)
        return True

    def is_captured(self, x, y, player, group, visited):
        if (x, y) in visited:
            return True  # すでにチェック済み
        visited.add((x, y))

        if self.board[x][y] == '.':
            return False  # 自由度がある
        if self.board[x][y] != player:
            return True  # 相手の石はブロックになる

        group.add((x, y))

        # 隣接する位置を再帰的にチェック
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if not self.is_captured(nx, ny, player, group, visited):
                    return False  # 自由度がある
        return True  # 自由度がない

    def remove_dead_stones(self):
        # 死石の除去と捕獲石のカウント
        dead_stones = {'B': set(), 'W': set()}
        for player in ['B', 'W']:
            opponent = 'W' if player == 'B' else 'B'
            for x in range(self.size):
                for y in range(self.size):
                    if self.board[x][y] == player:
                        if not self.is_alive(x, y, player, set()):
                            dead_stones[opponent].add((x, y))
        for player in ['B', 'W']:
            for x, y in dead_stones[player]:
                self.board[x][y] = '.'
            self.captured_stones[player] += len(dead_stones[player])

    def is_alive(self, x, y, player, visited):
        if (x, y) in visited:
            return False
        visited.add((x, y))

        # 二つの眼があるかをチェック
        eyes = self.count_eyes(x, y, player, set())
        if eyes >= 2:
            return True

        # 周囲に空きがあるかをチェック
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if self.board[nx][ny] == '.':
                    return True
                elif self.board[nx][ny] == player:
                    if self.is_alive(nx, ny, player, visited):
                        return True
        return False

    def count_eyes(self, x, y, player, visited):
        if (x, y) in visited:
            return 0
        visited.add((x, y))
        eyes = 0
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if self.board[nx][ny] == '.':
                    if self.is_eye(nx, ny, player, set()):
                        eyes += 1
                elif self.board[nx][ny] == player:
                    eyes += self.count_eyes(nx, ny, player, visited)
        return eyes

    def is_eye(self, x, y, player, visited):
        if (x, y) in visited:
            return True
        visited.add((x, y))
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                if self.board[nx][ny] != player:
                    return False
        return True

    def calculate_territory(self):
        territory = {'B': 0, 'W': 0}
        checked = set()
        for x in range(self.size):
            for y in range(self.size):
                if self.board[x][y] == '.' and (x, y) not in checked:
                    owners, area = self.find_owners(x, y, checked)
                    if len(owners) == 1:
                        owner = owners.pop()
                        territory[owner] += area
        return territory

    def find_owners(self, x, y, checked):
        owners = set()
        area = 0
        stack = [(x, y)]
        while stack:
            cx, cy = stack.pop()
            if (cx, cy) in checked:
                continue
            checked.add((cx, cy))
            area += 1
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = cx + dx, cy + dy
                if 0 <= nx < self.size and 0 <= ny < self.size:
                    if self.board[nx][ny] == '.':
                        stack.append((nx, ny))
                    else:
                        owners.add(self.board[nx][ny])
        return owners, area

    def has_valid_moves(self, player):
        for x in range(self.size):
            for y in range(self.size):
                if self.board[x][y] == '.' and self.is_valid_move(x, y, player):
                    return True
        return False

    def get_valid_moves(self, player):
        valid_moves = []
        for x in range(self.size):
            for y in range(self.size):
                if self.board[x][y] == '.' and self.is_valid_move(x, y, player):
                    valid_moves.append((x, y))
        return valid_moves

    def is_valid_move(self, x, y, player):
        # 仮に石を置いて、自殺手でないかチェックする
        self.board[x][y] = player
        is_valid = True
        if not self.check_and_capture_for_validation(x, y):
            is_valid = False
        self.board[x][y] = '.'  # 盤面を元に戻す
        return is_valid

    def check_and_capture_for_validation(self, x, y):
        player = self.board[x][y]
        opponent = 'W' if player == 'B' else 'B'
        captured_opponent_stones = set()

        # 相手の石をチェック
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size and self.board[nx][ny] == opponent:
                group = set()
                if self.is_captured(nx, ny, opponent, group, set()):
                    captured_opponent_stones.update(group)

        # 仮に相手の石を取り除く
        for gx, gy in captured_opponent_stones:
            self.board[gx][gy] = '.'

        # 自分の石が捕獲されるかチェック
        group = set()
        result = True
        if self.is_captured(x, y, player, group, set()):
            if len(captured_opponent_stones) == 0:
                result = False  # 自殺手は無効

        # 取り除いた相手の石を元に戻す
        for gx, gy in captured_opponent_stones:
            self.board[gx][gy] = opponent

        return result

    def play(self):
        # 盤面サイズの指定
        size_input = input("盤面のサイズを指定してください（9, 13, 19 など）[デフォルト: 9]: ")
        if size_input.isdigit():
            self.size = int(size_input)
            self.board = [['.' for _ in range(self.size)] for _ in range(self.size)]
        else:
            print("デフォルトのサイズ9でゲームを開始します。")
        while not self.game_over:
            if not self.has_valid_moves(self.current_player):
                print(f"プレイヤー {self.get_player_symbol(self.current_player)} は有効な手がありません。")
                self.end_game()
                break

            self.print_board()
            print(f"取った石：●={self.captured_stones['B']}、○={self.captured_stones['W']}")
            if self.current_player == 'B':
                try:
                    move = input(f"プレイヤー {self.get_player_symbol(self.current_player)}、手を入力してください（行 列）: ")
                    if move.lower() == 'quit':
                        print("ゲームを終了します。")
                        self.end_game()
                        break
                    x, y = map(int, move.strip().split())
                    if x < 1 or x > self.size or y < 1 or y > self.size:
                        print("無効な手です。範囲外です。")
                        continue
                    if not self.place_stone(x, y):
                        continue
                except ValueError:
                    print("無効な入力です。行と列の数字を入力してください。")
            else:
                self.model_move()

    def get_player_symbol(self, player):
        return '●' if player == 'B' else '○'

    def end_game(self):
        # 1. 黒石に囲まれた白の死石を黒が取り、白石に囲まれた黒の死石を白が取る
        self.remove_dead_stones()

        # 2. スコア計算用盤面の表示
        print("スコア計算用盤面：")
        self.print_board()

        # 3. 黒石に囲まれた陣地のマス目の数を数える
        # 4. 白石に囲まれた陣地のマス目の数を数える
        territory = self.calculate_territory()

        # 5. スコアの計算
        total_score = {
            'B': territory['B'] + self.captured_stones['B'],
            'W': territory['W'] + self.captured_stones['W']
        }

        # 6. スコアの表示
        print("取った石：")
        print(f"●: {self.captured_stones['B']} 個")
        print(f"○: {self.captured_stones['W']} 個")
        print("陣地：")
        print(f"●: {territory['B']} 目")
        print(f"○: {territory['W']} 目")
        print("最終得点：")
        print(f"●: {total_score['B']} 点")
        print(f"○: {total_score['W']} 点")

        # 7. 勝者の判定
        if total_score['B'] > total_score['W']:
            print("プレイヤー ● の勝ちです！")
        elif total_score['B'] < total_score['W']:
            print("プレイヤー ○ の勝ちです！")
        else:
            print("引き分けです！")
        self.game_over = True

if __name__ == "__main__":
    game = GoGame()
    game.play()



盤面のサイズを指定してください（9, 13, 19 など）[デフォルト: 9]: 5
    1  2  3  4  5
 1 ・ ・ ・ ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ・ ・ ・
 5 ・ ・ ・ ・ ・

取った石：●=0、○=0
プレイヤー ●、手を入力してください（行 列）: 1 1
    1  2  3  4  5
 1 ● ・ ・ ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ・ ・ ・
 5 ・ ・ ・ ・ ・

取った石：●=0、○=0
モデル（プレイヤー ○）が石を置きました: 4 3
    1  2  3  4  5
 1 ● ・ ・ ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ○ ・ ・
 5 ・ ・ ・ ・ ・

取った石：●=0、○=0
プレイヤー ●、手を入力してください（行 列）: 1
無効な入力です。行と列の数字を入力してください。
    1  2  3  4  5
 1 ● ・ ・ ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ○ ・ ・
 5 ・ ・ ・ ・ ・

取った石：●=0、○=0
プレイヤー ●、手を入力してください（行 列）: 1 3
    1  2  3  4  5
 1 ● ・ ● ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ○ ・ ・
 5 ・ ・ ・ ・ ・

取った石：●=0、○=0
モデル（プレイヤー ○）が石を置きました: 5 2
    1  2  3  4  5
 1 ● ・ ● ・ ・
 2 ・ ・ ・ ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ○ ・ ・
 5 ・ ○ ・ ・ ・

取った石：●=0、○=0
プレイヤー ●、手を入力してください（行 列）: 2 3
    1  2  3  4  5
 1 ● ・ ● ・ ・
 2 ・ ・ ● ・ ・
 3 ・ ・ ・ ・ ・
 4 ・ ・ ○ ・ ・
 5 ・ ○ ・ ・ ・

取った石：●=0、○=0
モデル（プレイヤー ○）が石を置きました: 5 1
    1  2  3  4  5
 1 ● ・ ● ・ ・
 2 ・ ・ ● ・ ・
 3 ・ ・ ・ ・ ・
 4 ・